In [ ]:
!pip install mapclassify

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import geopandas as gpd
import pandas as pd
import os
import matplotlib


# Feel free to use these funcitons, I developed them with the support of co-pilot

def split_geojson_by_size(file_path, max_size_mb, output_folder):
    # Load the GeoJSON file into a GeoDataFrame
    gdf = gpd.read_file(file_path)

    # Calculate the approximate size of each feature
    num_features = len(gdf)
    total_size_mb = os.path.getsize(file_path) / (1024 * 1024)
    size_per_feature_mb = total_size_mb*1.5 / num_features

    # Calculate the number of features per split based on max_size_mb
    features_per_split = int((max_size_mb / size_per_feature_mb)/2)

    # Split the GeoDataFrame into smaller GeoDataFrames
    split_gdfs = [gdf.iloc[i:i + features_per_split] for i in range(0, num_features, features_per_split)]

    # # Save each smaller GeoDataFrame to a separate GeoJSON file
    # for i, split_gdf in enumerate(split_gdfs):
    #     split_gdf.to_file(f"split_{i}.geojson", driver="GeoJSON")

    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Save each smaller GeoDataFrame to a separate GeoJSON file in the output folder
    for i, split_gdf in enumerate(split_gdfs):
        split_gdf.to_file(os.path.join(output_folder, f"split_{i}.geojson"), driver="GeoJSON")

def merge_geojson(file_paths):
    # Load each GeoJSON file into a GeoDataFrame and concatenate them
    gdfs = [gpd.read_file(file_path) for file_path in file_paths]
    merged_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

    return merged_gdf


In [ ]:
#
# large file downloaded from:https://geoportal.statistics.gov.uk/datasets/12baf1e6a44441208ffe5ba5ed063a68_0/explore?location=0.001591%2C177.510517%2C0.00
# saved in MyDrive
msoa =gpd.read_file('/content/drive/MyDrive/Middle_layer_Super_Output_Areas_December_2021_Boundaries_EW_BFC_V7_303696399389513507.geojson')

In [ ]:
#msoa.explore()

In [ ]:
msoa.head()

Option 1: Subset data the first time and save subsetted data as a new geojson. Work with this .geojson, but keep the code (commented out) that creates the geojson .file.

In [ ]:
# Uncomment these lines the first time you subset the data

# # Get MSOAs where name contains Southampton
# southampton = msoa.loc[msoa['MSOA21NM'].str.contains('Southampton'),:]
# # save Southampton as a .geojson
# southampton.to_file('/content/drive/MyDrive/Southampton.geojson')


# Comment out the above and just use this file every other time your run the code. Uplaod this file to GitHub
southampton =gpd.read_file('/content/drive/MyDrive/Southampton.geojson')

In [ ]:
southampton.explore()

Option 2: If you need to work with a large file every time, save it in your drive. YOu can then split the file into smaller files, small enough to upload to GitHub

In [ ]:

# Use the split_geojson_by_size function
# Split the large GeoJSON file into smaller files with a maximum size of 24 MB each
# the split files will be in the 'msoas_split_geojson' folder in your drive
split_geojson_by_size("/content/drive/MyDrive/Middle_layer_Super_Output_Areas_December_2021_Boundaries_EW_BFC_V7_303696399389513507.geojson", max_size_mb=24, output_folder='/content/drive/MyDrive/msoas_split_geojson')



In [ ]:
# Merge the smaller GeoJSON files back into one GeoDataFrame
#file_paths = [f"split_{i}.geojson" for i in range(len([f for f in os.listdir() if 'split_' in f]))]

file_paths = [f"split_{i}.geojson" for i in range(len([f for f in os.listdir('/content/drive/MyDrive/msoas_split_geojson') if 'split_' in f]))]
merged_gdf = merge_geojson(file_paths)

# Look at the merged GeoDataFrame
merged_gdf.head()

You can then upload these smaller files to GitHub